In [1]:
import unicodedata
import re
import pandas as pd

import numpy as np

train = pd.read_csv(r'Data/train_final.csv')
dev = pd.read_csv(r'Data/dev_final.csv')

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z0-9?.!,¿|]+", " ", w)
    w = w.strip()
    w = '<start> ' + w + ' <end>'
    return w


import tensorflow as tf

# Convert sequences to tokenizers
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
    # tensor = tf.ragged.constant(tensor)
    # print(tensor)
    return tensor, lang_tokenizer


# Load the dataset
def load_dataset(number):
    # Create dataset (targ_lan = English, inp_lang = French)
    inp_lang = (train['paragraph']+" | "+train['question']).copy()
    targ_lang = (train['answer']).copy()
    act_inp = []
    act_out = []
    ep = number
    if number == 0:
        ep = len(inp_lang)
    for i in range(ep):
        if(len(str(targ_lang[i]).split(' '))==1):
            act_inp.append(preprocess_sentence(str(inp_lang[i])))
            act_out.append(preprocess_sentence(str(targ_lang[i])))
    
    inp_lang = (dev['paragraph']+" | "+dev['question']).copy()
    targ_lang = (dev['answer']).copy()
    if number == 0:
        ep = len(inp_lang)
    for i in range(ep):
        if(len(str(targ_lang[i]).split(' '))==1):
            act_inp.append(preprocess_sentence(str(inp_lang[i])))
            act_out.append(preprocess_sentence(str(targ_lang[i])))
            
    input_tensor, inp_lang_tokenizer = tokenize(act_inp)
    target_tensor, targ_lang_tokenizer = tokenize(act_out)
    
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [2]:
number_of_instances = 0

input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer = load_dataset(number_of_instances)
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

print(max_length_targ, max_length_inp )

from sklearn.model_selection import train_test_split

input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)


embeddings_index = {}
f = open('glove.42B.300d.txt', encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

inp_embedding_matrix = np.zeros((len(inp_lang_tokenizer.word_index) + 1, 300))
targ_embedding_matrix = np.zeros((len(targ_lang_tokenizer.word_index) + 1, 300))

for word, i in inp_lang_tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        inp_embedding_matrix[i] = embedding_vector
        
for word, i in targ_lang_tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        targ_embedding_matrix[i] = embedding_vector

print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

3 579
17116 17116 4280 4280


In [3]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 1
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 50
vocab_inp_size = len(inp_lang_tokenizer.word_index) + 1
vocab_tar_size = len(targ_lang_tokenizer.word_index) + 1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset_test = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)

In [4]:
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
# Size of input and target batches
example_input_batch, example_target_batch = next(iter(dataset))
print(example_input_batch.shape, example_target_batch.shape)

# Encoder class
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim,  weights=[inp_embedding_matrix], trainable=False)
        self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
    
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))


# Attention Mechanism
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights
    
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))


# Decoder class
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[targ_embedding_matrix], trainable=False)
        self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights
    
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))


# Initialize optimizer and loss functions
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy( from_logits=True, reduction='none')

# Loss function
def loss_function(real, pred):
    # Take care of the padding. Not all sequences are of equal length.
    # If there's a '0' in the sequence, the loss is being nullified
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    fontdict = {'fontsize': 14}
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    plt.show()
    return
    
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = preprocess_sentence(sentence)
    inputs = [inp_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['<start>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang_tokenizer.index_word[predicted_id] + ' '
        if targ_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence, attention_plot

def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))
    return

def get_data(element):
    question = ''
    answer = ''
    for k in element[0].numpy():
        if k!=0:
            question = question+ inp_lang_tokenizer.index_word[k] + ' '
    for k in element[1].numpy():
        if k!=0:
            answer = answer+ targ_lang_tokenizer.index_word[k] + ' '
    return question, answer


import os
checkpoint_dir = './training_checkpoints1'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)


@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

(1, 579) (1, 3)
Encoder output shape: (batch size, sequence length, units) (1, 579, 50)
Encoder Hidden state shape: (batch size, units) (1, 50)
Attention result shape: (batch size, units) (1, 50)
Attention weights shape: (batch_size, sequence_length, 1) (1, 579, 1)
Decoder output shape: (batch_size, vocab size) (1, 9720)


In [ ]:
#Only RUN FOR TRAINING
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
import time
EPOCHS = 500

for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
    if (epoch + 1) % 3 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)
    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

2022-03-09 19:49:18.200621: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1 Batch 0 Loss 0.4245
Epoch 1 Batch 100 Loss 2.7458
Epoch 1 Batch 200 Loss 1.8149
Epoch 1 Batch 300 Loss 0.0345
Epoch 1 Batch 400 Loss 4.9575
Epoch 1 Batch 500 Loss 1.9590
Epoch 1 Batch 600 Loss 2.3455
Epoch 1 Batch 700 Loss 2.8050
Epoch 1 Batch 800 Loss 1.1944
Epoch 1 Batch 900 Loss 2.5513
Epoch 1 Batch 1000 Loss 1.0732
Epoch 1 Batch 1100 Loss 0.4388
Epoch 1 Batch 1200 Loss 0.0794
Epoch 1 Batch 1300 Loss 0.1432
Epoch 1 Batch 1400 Loss 4.6689
Epoch 1 Batch 1500 Loss 0.7886
Epoch 1 Batch 1600 Loss 0.1795
Epoch 1 Batch 1700 Loss 0.7464
Epoch 1 Batch 1800 Loss 0.3038
Epoch 1 Batch 1900 Loss 1.1206
Epoch 1 Batch 2000 Loss 4.2130
Epoch 1 Batch 2100 Loss 2.3105
Epoch 1 Batch 2200 Loss 2.1616
Epoch 1 Batch 2300 Loss 2.5812
Epoch 1 Batch 2400 Loss 2.5249
Epoch 1 Batch 2500 Loss 1.7243
Epoch 1 Batch 2600 Loss 1.3915
Epoch 1 Batch 2700 Loss 2.7068
Epoch 1 Batch 2800 Loss 0.7097
Epoch 1 Batch 2900 Loss 6.4095
Epoch 1 Batch 3000 Loss 1.7551
Epoch 1 Batch 3100 Loss 1.0452
Epoch 1 Batch 3200 L